# Final Project: Automated solutions for algorithmic bias, Python Portion
##### by: Cheng-Yu (Ben) Chiang from MATH 157 Winter 2022

## Resource used: 
- https://www.reuters.com/article/us-amazon-com-jobs-automation-insight/amazon-scraps-secret-ai-recruiting-tool-that-showed-bias-against-women-idUSKCN1MK08G
- https://en.wikipedia.org/wiki/Disparate_impact#:~:text=Disparate%20impact%20in%20United%20States,or%20landlords%20are%20formally%20neutral
- https://github.com/Trusted-AI/AIF360
- https://dl.acm.org/doi/pdf/10.1145/3278721.3278779
- https://aif360.readthedocs.io/en/stable/modules/generated/aif360.datasets.BinaryLabelDataset.html#aif360.datasets.BinaryLabelDataset

## What is algorithmic bias? 
- Algorithmic bias refers error made by computer algorithms that create unfair outcomes, usually results in privileging one particular group of users over the other.
- It can emerge from intentional / unintentional factors such as the design of the algorithm, biased sampling of the data, etc.
- Algorithms have a strong social and political impact on our lives: presidential polls, search engines, credit score, advertisement, etc.

## Example: Amazon's AI recruiting tool is biased against women
- One of the examples of algorithmic bias is the famous case of amazon's AI recruiting tool biasing against women. Amazon, like many big tech companies, has been using automated tools to recruit top talents from the huge pool of candidates that they receive applications from. In an ideal situation, all candidates will be judged fairly based off their abilities.
- However, in 2015, Amazon discovered that its hiring algorithm for software developers are heavily biased against women. 
- This is because the machine learning algorithm were trained to observe pattern from previous successful hires and most of these data points come from men.
- Eventually, the team that was in charge of this project was disbanded.
- Highlights the impact in algorithmic bias and the difficulty in solving such problems. 

## The challenge
- Bias is a human nature
- How can we make sure our algorithms does not amplify our inherent bias?

### Data input and cleaning

In [1]:
import pandas as pd
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.datasets import BinaryLabelDataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

pd.set_option("display.precision", 5)


col_names = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
             "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
             "Hours per week", "Country", "Target"]

train = pd.read_csv("raw_dataset/adult.data", skiprows=0, names=col_names, header=None)
test = pd.read_csv("raw_dataset/adult.test", skiprows=1, names=col_names, header=None)

df = pd.concat([train, test]).reset_index(drop=True)
df = df.drop(['Education', 'fnlwgt', 'Country'], axis=1)
df['Sex'] = df['Sex'].str.strip()
df['Target'] = df['Target'].str.strip()

df['Target'] = df['Target'].replace(['<=50K', '<=50K.'], 0)
df['Target'] = df['Target'].replace(['>50K', '>50K.'], 1)
df.to_csv("parsed_dataset/complete_data.csv")


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


2022-03-18 02:24:44.916791: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-18 02:24:44.916828: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def summarize(df, gender: str):
    gender_df = df[df['Sex'] == gender]
    print("="*30)
    print(gender, "<=50K")
    print("count:", sum(gender_df['Target'] == 0))
    print("prob: ",  sum(gender_df['Target'] == 0) / len(gender_df))
    print("="*30)
    print(gender, ">50K")
    print("count:", sum(gender_df['Target'] == 1))
    print("prob: ",  sum(gender_df['Target'] == 1) / len(gender_df))
    return gender_df
     
# key should be "Male" or "Female"
male = summarize(df, 'Male')
female = summarize(df, 'Female')

Male <=50K
count: 22732
prob:  0.6962327718223583
Male >50K
count: 9918
prob:  0.3037672281776417
Female <=50K
count: 14423
prob:  0.8907485177865613
Female >50K
count: 1769
prob:  0.10925148221343874


As you can see, there are more male represented in the >50k region. Based on the dataset, about 3 in 10 male adult earns more than 50K. On the other hand, only 1 in 10 female adults earn more than 50K. 
Clearly, something is wrong. Whether or not this is an accurate representation of male and female in the workforce is up for debate. However, as an engineer / scientist our job is to make sure that the algorithm that we designed isn't biased against either group for whatever reason.

## Measuring Bias

To measure bias, we will not introduce some common metrics. Including Statistical Parity Difference, Equal Opportunity Difference, Average Odds Difference, and Disparate Impact Ratio. Each metric can be interpreted individually or together to give us a insight on whether the dataset / model is biased and in what ways. 

The first 3 will be introduced in the julia portion of the project and the last metric, disparate impact ratio, will be introduced after this section. 

#### **If you are reading this presentation instead of being presented to, please head to the Julia portion of this presentation**

<br></br>
### Disparate impact metric
Now, in order to see if our method is working, we need a way to measure the bias in our dataset / population. One good way to do it is using **disparate impact** ratio.

Disparate impact ratio is a metric formally defined by the United States labor law by to evaluate fairness based on a predefined positive output. The general equation for the disparate impact is
$$
\frac{P(Y=1|\text{unprivileged})}{P(Y=1|\text{privileged})}
$$
We define the positive output as >50K income (Y = 1) and assume that the unprivileged group to be female based on its significantly higher <=50K income probability compared to male. Conversely, we assume that male is the privileged group.

So, the disparate impact metric becomes: 

$$
\frac{P(\text{income}>\text{50K}|\text{women})}{P(\text{income}>\text{50K}|\text{men})}
$$

In [3]:
(sum(female['Target'] == 1) / len(female)) / (sum(male['Target'] == 1) / len(male))

0.3596552625800337

## Solution 1: Reweighting (Preprocessing)
One way to mitigate the bias is by modifying the data before the training has taken place.
A good method that is commonly used is reweighting. This refers to giving weights to different row features based on their privilege status and ourcome. 

Below, we will layout the foundation of such method by hand calculating the weights that should be given to different classes / outcomes.

### The 4/5 Rule
The typical standard for an acceptable disparate impact rate that is the four-fifths rule:
- if the unprivileged group gets a positive outcome (income > 50K) less than 80% of their proportion of the privileged group, then this counts as a disparate impact violation
- this means that our calculation results shown above should be > 0.8 for it to not be a violation
- our disparate impact ratio of `0.358` means that there is clearly an bias against women in terms of income based on the sampling in this dataset

## Risk of Training with Biased data

The risk of training with biased data lies in the possibility of the model amplifying the mistake. 

As a machine learning model, it's goal is to maximum its accuracy. Therefore, it will be naturally intended to lean towards the privileged group. 

An example to understand this is: 
- say we have a dataset with 10000 samples, 1000 of them has gender = female and the rest is marked male.
- If our model were to predict with 0.5 probability (randomly predicting), then it's expected accuracy is only 0.9 * 0.5 + 0.1 * 0.5 = 0.5 (50%)
- However, if our model predicts male all the time, then it will be right 90% of the time. 
- Therefore, the model is inclined to be biased towards male simply because it is more well represented in the dataset. 

Below we provide a real example of how classifying using biased data can lead to worse biasing.

In [4]:
categorical_df = pd.get_dummies(df)
labels = categorical_df['Target']
features = categorical_df.drop(['Target'], axis=1)

scaler = StandardScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

x_train, x_test, y_train, y_test = train_test_split(scaled_df, labels, test_size=0.3, random_state=42)

In [5]:
reg = LogisticRegression()
reg.fit(x_train, y_train)

LogisticRegression()

In [6]:
def test_model(x, y, df):
    orig_matching_df = df.iloc[x.index, :]
    pred = reg.predict(x)
    probs = reg.predict_proba(x)
    print("The mean accuracy of the model is:", round(reg.score(x, y), 5))
    data = {'target': y, 'prediction': pred ,'class_0_prob': probs[:, 0], 'class_1_prob': probs[:, 1], 'gender': orig_matching_df['Sex']}
    result_df = pd.DataFrame(data)
    return result_df

result_df = test_model(x_test, y_test, df)
result_df.head()

The mean accuracy of the model is: 0.85006


,target,prediction,class_0_prob,class_1_prob,gender
7762,0,0,0.99531,0.00469,Male
23881,0,0,0.99826,0.00174,Female
30507,0,0,0.99665,0.00335,Male
28911,0,0,0.99555,0.00445,Female
19484,0,0,0.97298,0.02702,Male


In [7]:
female = result_df[result_df['gender'] == 'Female']
print('Model predicts', round(sum(female['prediction'] == 1) / len(female), 5) * 100, '% of female earn >50K')
male = result_df[result_df['gender'] == 'Male']
print('Model predicts', round(sum(male['prediction'] == 1) / len(male), 5) * 100, '% of male earn >50K')

print("Predicted disparate impact ratio is:", (sum(female['prediction'] == 1) / len(female)) / (sum(male['prediction'] == 1) / len(male)))

result_df.to_csv("parsed_dataset/biased_results.csv")

Model predicts 7.034 % of female earn >50K
Model predicts 25.176 % of male earn >50K
Predicted disparate impact ratio is: 0.27939670399997324


Clearly, the classifier has worsened the bias in the dataset. It will be critical that we mitigate this before using it in any way.

## Assigning weights to different classes
To rectify this bias what we can do is apply a weight for each class depending on whether it its unprivileged and privileged. Intuitively, we would want to give the privileged class (male) less weighting and the less privileged class (female) a heavier weight. 

This can be formulated as the following equation: 
$$
W = \frac{N_{\text{unprivileged}}N_{\text{positive}}}{N_{\text{all}}N_{\text{positive unprivileged}}}
$$

In [8]:
male = df[df['Sex'] == 'Male']
female = df[df['Sex'] == 'Female']

pos_unpri = (len(female) * sum(df['Target'] == 1)) / (len(df) * sum(female['Target'] == 1))
neg_unpri = (len(female) * sum(df['Target'] == 0)) / (len(df) * sum(female['Target'] == 0))

pos_pri = (len(male) * sum(df['Target'] == 1)) / (len(df) * sum(male['Target'] == 1))
neg_pri = (len(male) * sum(df['Target'] == 0)) / (len(df) * sum(male['Target'] == 0))

print('weight for positive underprivileged class:', round(pos_unpri, 5))
print('weight for negative underprivileged class:', round(neg_unpri, 5))
print('weight for position privileged class:', round(pos_pri, 5))
print('weight for negative privileged class:', round(neg_pri, 5))

weight for positive underprivileged class: 2.19019
weight for negative underprivileged class: 0.85402
weight for position privileged class: 0.78771
weight for negative privileged class: 1.09262


How can we explain these results intuitively?
- higher weight for positive underprivileged class: we want to boost the positive features of underprivileged class
- lower weight for negative underprivileged class: we want to reduce the negative features of underprivileged class
- lower weight for positive privileged class: we want to reduce the amount of positive for the privileged class
- slightly higher weight for negative privileged class: we want to slightly increase the effect of negative attributes to the privileged class

## Reweighing the dataset and retrain the model 
Now, after we obtained the weights, what we want to do is to apply the calculated weights to the entire dataset. To implementing this, we can simply use the provided function from `aif360` library `Reweighing`. 
A brief introduction to the `aif360` library: 
- Developed by IBM's research team Trusted AI, it is an opensource toolkit for examining, reporting, and mitigating discrimination and bias in machine learning models.
- The package includes comprehensive set of metrics for biases and models and a collection of algorithms to mitigate bias in dataset and machine learned models. 
- Using the package, we can load common datasets such as the Adult Census Income Dataset, Bank marketing Dataset, German credit Dataset, etc.
- The package splits the algorithms into 3 main types:
    1. Preprocessing: modifying the dataset before training
    2. Inprocessing: modifying the training process to ensure fairness
    3. Postprocessing: calibrate, reject, or equalize outcomes after training has been done
- In this example, I will only demonstrate a few examples of the functionality that it provides. To learn more, go to https://aif360.mybluemix.net/data for an interactive demo

Reweighting is an algorithm that belongs to the class of preprocessing algorithms because it modifies the dataset before training happens.

Below, we demonstrate how `Reweighing` can be used to mitigate bias in the dataset and make the model less prone to be biased.

In [9]:
# take out gender column so it doesn't get transformed
gender_df = df['Sex'].copy()
gender_df[gender_df == 'Female'] = 1
gender_df[gender_df == 'Male'] = 0

# drop the unnecessary columns
new_df = pd.get_dummies(df).drop(['Sex_Male', 'Sex_Female'], axis=1)
new_df['Gender'] = gender_df

In [10]:
# convert encoded data into required aif format
BLD = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=new_df, label_names=['Target'], protected_attribute_names=['Gender'])

privileged_groups = [{'Gender': 1}]
unprivileged_groups = [{'Gender': 0}]

# reweight the dataset using built-in function from aif360
RW = Reweighing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}])
di = RW.fit_transform(BLD)
print(di.instance_weights, "\n", len(di.instance_weights))

data = {'gender': df['Sex'], 'target': new_df['Target'], 'weight': di.instance_weights}
pd.DataFrame(data).head(15)

[1.09262055 1.09262055 1.09262055 ... 1.09262055 1.09262055 0.78771422] 
 48842


,gender,target,weight
0,Male,0,1.09262
1,Male,0,1.09262
2,Male,0,1.09262
3,Male,0,1.09262
4,Female,0,0.85402
5,Female,0,0.85402
6,Female,0,0.85402
7,Male,1,0.78771
8,Female,1,2.19019
9,Male,1,0.78771


As we can see here, the previous weights that we calculated are now applied to the entire dataset based on which of the four classes that it is in. 

In [11]:
di_features = pd.DataFrame(di.features, columns=di.feature_names).reset_index(drop=True)

# scale the features
scaler = StandardScaler()
scaled_features = pd.DataFrame(scaler.fit_transform(di_features), columns=di_features.columns)

# split the dataset into train / test sets
x_train, x_test, y_train, y_test = train_test_split(scaled_features, di.labels.flatten(), test_size=0.3, random_state=42)

In [12]:
reg.fit(x_train, y_train, sample_weight=di.instance_weights[x_train.index])

LogisticRegression()

In [13]:
result_df = test_model(x_test, y_test, df)
result_df

The mean accuracy of the model is: 0.84324


,target,prediction,class_0_prob,class_1_prob,gender
7762,0.0,0.0,0.99651,0.00349,Male
23881,0.0,0.0,0.99573,0.00427,Female
30507,0.0,0.0,0.99773,0.00227,Male
28911,0.0,0.0,0.98880,0.01120,Female
19484,0.0,0.0,0.98053,0.01947,Male
...,...,...,...,...,...
15938,0.0,0.0,0.65240,0.34760,Male
27828,0.0,0.0,0.99038,0.00962,Female
28449,0.0,0.0,0.94207,0.05793,Female
5647,0.0,0.0,0.98255,0.01745,Male


In [14]:
female = result_df[result_df['gender'] == 'Female']
print('Model predicts', round(sum(female['prediction'] == 1) / len(female), 5) * 100, '% of female earn >50K')

male = result_df[result_df['gender'] == 'Male']
print('Model predicts', round(sum(male['prediction'] == 1) / len(male), 5) * 100, '% of male earn >50K')

print("Predicted disparate impact ratio is:", (sum(female['prediction'] == 1) / len(female)) / (sum(male['prediction'] == 1) / len(male)))

Model predicts 12.073 % of female earn >50K
Model predicts 20.488 % of male earn >50K
Predicted disparate impact ratio is: 0.5892766989037003


## Solution 2: Adversarial Debiasing
- An inprocessing technique which learns two classifiers in simultaneously to ensure that protected attributes such as gender and race cannot be used to produce biased models. 
- One classifier's job is to maximize prediction accuracy or minimize loss
- The other classifier tries to determine the protected attribute from the prediction produced by the first classifier. 
- Goal: to minimize the adversary's ability to correctly identify the protected attribute while reducing the error of the original classification problem. 
- Intuitively: if the difference in results between unprivileged and privileged group cannot be discerned, then we have achieved our goal of removing bias from the training process. 

In [15]:
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
import tensorflow.compat.v1 as tf

tf.disable_eager_execution()
sess = tf.Session()

2022-03-18 02:25:01.762683: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-18 02:25:01.763252: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-18 02:25:01.763786: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (project-38c1918c-f051-4428-8847-997d00b4b0c8): /proc/driver/nvidia/version does not exist
2022-03-18 02:25:01.771621: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
filtered_df = df[['Age', 'Workclass', 'Education-Num', 'Occupation', 'Sex', 'Capital Gain', 'Capital Loss',
       'Hours per week', 'Target']]

gender_df = filtered_df['Sex'].copy()
gender_df[gender_df == 'Female'] = 1
gender_df[gender_df == 'Male'] = 0

# drop the unnecessary columns
new_df = pd.get_dummies(filtered_df).drop(['Sex_Male', 'Sex_Female'], axis=1)
new_df['Gender'] = gender_df
new_df = new_df.drop("Workclass_ ?", axis=1)


BLD = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=new_df, label_names=['Target'], protected_attribute_names=['Gender'])

In [17]:
privileged_groups = [{'Gender': 1}]
unprivileged_groups = [{'Gender': 0}]

debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess,
                          num_epochs=100)

In [18]:
debiased_model.fit(BLD)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 26.520050; batch adversarial loss: 0.721980


epoch 0; iter: 200; batch classifier loss: 6.575389; batch adversarial loss: 0.672951


epoch 1; iter: 0; batch classifier loss: 15.754431; batch adversarial loss: 0.648825


epoch 1; iter: 200; batch classifier loss: 8.892447; batch adversarial loss: 0.636596


...

epoch 97; iter: 200; batch classifier loss: 0.771629; batch adversarial loss: 0.591383


epoch 98; iter: 0; batch classifier loss: 0.501463; batch adversarial loss: 0.580412


epoch 98; iter: 200; batch classifier loss: 0.404070; batch adversarial loss: 0.653818


epoch 99; iter: 0; batch classifier loss: 0.693034; batch adversarial loss: 0.652979


epoch 99; iter: 200; batch classifier loss: 0.492253; batch adversarial loss: 0.604289


We can use the built-in functionality of aif360 to measure how our model is performing

In [19]:
from aif360.metrics import BinaryLabelDatasetMetric

debiased_ds = debiased_model.predict(BLD)
metric = BinaryLabelDatasetMetric(debiased_ds, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric.mean_difference())

metric = BinaryLabelDatasetMetric(BLD, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups = 0.093089
Difference in mean outcomes between unprivileged and privileged groups = 0.194516


We can calculate mean difference easily using pandas as well:  

In [21]:
sum(male['target']) / len(male) - sum(female['target']) / len(female)

0.19112980418681333

However, this is rather a crude metric, let's apply some of the metrics that we discussed earlier in the julia script to see how our model performs compared to our previous ones.

Thankfully, aif360 has built-in functionality of these metrics, so we don't have to hand write them like before. We can simply use the methods provided in `from aif360.metrics`, specifically `ClassificationMetric` directly. 

In [24]:
from aif360.metrics import ClassificationMetric

classification_metrics = ClassificationMetric(BLD, debiased_ds, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print(classification_metrics.accuracy())
print(classification_metrics.average_odds_difference())
print(classification_metrics.equal_opportunity_difference())

0.8312722656729864
0.044071753083786636
0.08035729043732376
